In [26]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [27]:
def pull_from_csv(csv_file, row=3, header=1):
        lwp_ind = str(csv_file).lower().find('lwp')
        lwp = str(csv_file)[lwp_ind:lwp_ind+14]
        df = pd.read_csv(csv_file, header=[header], on_bad_lines='skip')
        try:
            return [lwp,*df.iloc[row].name]
        except:
            return [lwp,*df.iloc[row]]
        
        
        
        
        
def met_ratio(df, up, down, error_method='Cramer', error_label=' Error (SD)', output='both'):
    if not isinstance(up,list):
        up = [up]
    if not isinstance(down,list):
        down = [down]    
   
    up_err_name = [f'{item}{error_label}' for item in up]
    down_err_name = [f'{item}{error_label}' for item in down]
    for item in np.concatenate([up_err_name, down_err_name]).flat:
        if item not in df.columns:
            raise KeyError(f'Cannot find "{item}" in the DataFrame. Please check the given name is correct.')
    
    up_amp = df[up].astype(float).sum(axis=1)
    down_amp = df[down].astype(float).sum(axis=1)
    if error_method.lower() == 'cramer':
        up_err = (df[up_err_name].astype(float)**2).sum(axis=1) ** 0.5
        down_err = (df[down_err_name].astype(float)**2).sum(axis=1) ** 0.5
    elif error_method.lower()=='narrow':
        up_err = df[up_err_name].astype(float).max(axis=1) 
        down_err = df[down_err_name].astype(float).max(axis=1) 
    else:
        raise KeyError('Please choose "Cramer" or "Narrow" for the error_method')
    
    ratio = up_amp / down_amp
    ratio_err = ratio * ((up_err/up_amp)**2 + (down_err/down_amp)**2)**0.5 
    
    if len(up)>1:
        up_label =f'({"+".join(up)})'
    else:
        up_label = f'{"+".join(up)}'
    if len(down)>1:
        down_label = f'({"+".join(down)})'
    else:
        down_label = f'{"+".join(down)}'    
    col_name = f'{up_label}/{down_label}'   
    col_error_name = f'{col_name} Error (SD) {error_method}'
    
    if output=='both':
        return pd.DataFrame({col_name: ratio, col_error_name: ratio_err})
    elif output=='ratio':
        return pd.DataFrame({col_name: ratio})      
    elif output=='error':
        return pd.DataFrame({col_error_name: ratio_err})   
    else:
        raise KeyError('Please choose "ratio", "error" or "both" (default) as output')

In [28]:
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY')
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/Ellie')
data_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA')

basis_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine')

sdat_files = [f for f in sorted(data_folder.rglob('*')) if ("act.sdat" in f.name.lower() and "press_35".lower() in f.name.lower())]
for file in tqdm(sdat_files):
    ref = Path(f'{str(file)[0:-8]}ref.SDAT')
    csv = Path(f'{str(file)[0:-5]}_PRESS_35_WM.csv')
    pdf = Path(f'{str(file)[0:-4]}pdf')
    # print(csv)
    if not csv.is_file():
        # command = f'tarquin --rows 3 --cols 2 --input {file} --output_pdf {pdf} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        # command = f'tarquin --rows 3 --cols 2 --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        command = f'tarquin --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'

        print(f'{command}\n') 
        os.system(command)
        

100%|██████████| 50/50 [00:00<00:00, 13058.23it/s]


In [29]:
csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("press_35" in f.name.lower() and ".csv" in f.name.lower())]



rows_to_process = {
                   'WM': 0,
                   }

for label,row_num in rows_to_process.items():
    data_list = []
    for file in csv_files:
        data_list.append([*pull_from_csv(file,row=row_num),*pull_from_csv(file,row=row_num+3),*pull_from_csv(file,header=6,row=row_num+1)])
    # print(f'{data_list=}')
    my_columns = list(pd.read_csv(file, header=[1], on_bad_lines='skip'))
    my_errors = [item+' Error (SD)' for item in my_columns]
    my_diagnostics = pull_from_csv(file,row=0, header=6)    
    df = pd.DataFrame(data_list, columns=['LWP', *my_columns, 'Error for LWP', *my_errors,*my_diagnostics ])
    df.to_excel(f'{label}.xlsx', index=False)
    
    #now calculate ratios
    df_L2N = met_ratio(df,['Lac', 'Threonine'], 'TNAA')
    df_L2N_narrow_error = met_ratio(df,['Lac', 'Threonine'], 'TNAA', error_method='Narrow', output='error')
    df_N2C = met_ratio(df,'TNAA', 'TCho')
    df_C2C = met_ratio(df,'TCho', 'Cr')
    df_N2Cr = met_ratio(df,'TNAA', 'Cr')


    # df_ratio = pd.concat((df_L2N, df_N2C, df_C2C, df_N2C, df), axis=1)
    df_ratio = pd.concat((df_L2N,df_L2N_narrow_error, df_N2C, df_C2C, df_N2Cr, df), axis=1)

    df_ratio.set_index('LWP').to_excel(f'{label}_RATIOS_PRESS_INSPIRE.xlsx')



ValueError: 105 columns passed, passed data had 107 columns

In [ ]:
dfbb = met_ratio(df,'TNAA', 'TCho', error_method='Narrow', output='error')
dfbb


In [24]:
my_columns

['Row',
 'Col',
 'Slice',
 '-CrCH2',
 'Ala',
 'Asp',
 'Cr',
 'GABA',
 'GPC',
 'Glc',
 'Gln',
 'Glu',
 'Gua',
 'Ins',
 'Lac',
 'Lip09',
 'Lip13a',
 'Lip13b',
 'Lip20',
 'MM09',
 'MM12',
 'MM14',
 'MM17',
 'MM20',
 'NAA',
 'NAAG',
 'PCh',
 'Scyllo',
 'Tau',
 'TNAA',
 'TCho',
 'Glx',
 'TLM09',
 'TLM13',
 'TLM20']

In [25]:
my_columns

['Row',
 'Col',
 'Slice',
 '-CrCH2',
 'Ala',
 'Asp',
 'Cr',
 'GABA',
 'GPC',
 'Glc',
 'Gln',
 'Glu',
 'Gua',
 'Ins',
 'Lac',
 'Lip09',
 'Lip13a',
 'Lip13b',
 'Lip20',
 'MM09',
 'MM12',
 'MM14',
 'MM17',
 'MM20',
 'NAA',
 'NAAG',
 'PCh',
 'Scyllo',
 'Tau',
 'TNAA',
 'TCho',
 'Glx',
 'TLM09',
 'TLM13',
 'TLM20']